## Optimization using gradient descent

This notebook allows one to do conduct uni- and multivariate linear fits using the gradient descent method. 


In [1]:
%matplotlib widget

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
#from mpltools import color as colors

In [2]:
data_range = np.random.RandomState(1) #make up some fake data for testing
fakex = 10 * data_range.rand(50)
fakey = 2 * fakex - 5 + data_range.randn(50)


!rm -rf ./data/housing*
!wget -P ./data https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv
    
#get some data for testing
!wget -P ./data https://datahub.io/core/global-temp/r/annual.csv

--2021-05-25 18:04:14--  https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘./data/housing.csv’

housing.csv         100%[===================>]   1.36M  3.43MB/s    in 0.4s    

2021-05-25 18:04:14 (3.43 MB/s) - ‘./data/housing.csv’ saved [1423529/1423529]

--2021-05-25 18:04:14--  https://datahub.io/core/global-temp/r/annual.csv
Resolving datahub.io (datahub.io)... 104.21.40.221, 172.67.157.38
Connecting to datahub.io (datahub.io)|104.21.40.221|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://pkgstore.datahub.io/core/global-temp/annual_csv/data/a26b154688b061cdd04f1df36e4408be/annual_csv.csv [fol

In [47]:
year, tempa = np.loadtxt('./data/annual.csv', skiprows=1, delimiter=",", usecols=(1,2), unpack=True)
year = np.array([int(y) for y in year])/np.max(year); tempa=np.array(tempa)+1

In [48]:
year = 1 + (year - np.mean(year))/(np.max(year)-np.min(year))

In [49]:
year

array([1.5       , 1.5       , 1.49264706, 1.49264706, 1.48529412,
       1.48529412, 1.47794118, 1.47794118, 1.47058824, 1.47058824,
       1.46323529, 1.46323529, 1.45588235, 1.45588235, 1.44852941,
       1.44852941, 1.44117647, 1.44117647, 1.43382353, 1.43382353,
       1.42647059, 1.42647059, 1.41911765, 1.41911765, 1.41176471,
       1.41176471, 1.40441176, 1.40441176, 1.39705882, 1.39705882,
       1.38970588, 1.38970588, 1.38235294, 1.38235294, 1.375     ,
       1.375     , 1.36764706, 1.36764706, 1.36029412, 1.36029412,
       1.35294118, 1.35294118, 1.34558824, 1.34558824, 1.33823529,
       1.33823529, 1.33088235, 1.33088235, 1.32352941, 1.32352941,
       1.31617647, 1.31617647, 1.30882353, 1.30882353, 1.30147059,
       1.30147059, 1.29411765, 1.29411765, 1.28676471, 1.28676471,
       1.27941176, 1.27941176, 1.27205882, 1.27205882, 1.26470588,
       1.26470588, 1.25735294, 1.25735294, 1.25      , 1.25      ,
       1.24264706, 1.24264706, 1.23529412, 1.23529412, 1.22794

In [50]:
housing_data = pd.read_csv('./data/housing.csv')

In [51]:
#housing_data

In [52]:
housing_data = housing_data[housing_data.median_house_value != 500001] #a bunch of values are set to this
housing_data = housing_data.sample(n=50) #smaller sample
housing_data.median_house_value = housing_data.median_house_value/100000

In [53]:
#housing_data.median_house_value.max()

In [54]:
#housing_data

In [55]:
def gradient_descent(x, y, gamma, l, theta_init = None):
    '''
    theta = Coordinates of the line I'm fitting. list or array
    gamma = Step size: Currently a constant. Later I will add an adaptive method to calculate it
    X = list or array. X values of data I am fitting to.
    y list or array. Y values of the data I am fitting to.
    l = polynomial order of fit
    '''
    n = len(x)

    #create X matrix
    X_matrix = np.zeros((n,l+1))
    if theta_init is not None:
        theta = theta_init
    else:
        theta = np.zeros(l+1)
    
    for i in range(n):
        for j in range(l+1):
            X_matrix[i,j] = x[i]**j
   
    X_matrix_T = X_matrix.T #transpose of X

    #now do it
    k = 1
    kmax = 500
    store_theta = []
    gamma_X_matrix_T = gamma*X_matrix_T
    epsilon = 0.0001
    stop_condition = False
    while not stop_condition:
        store_theta.append(theta)
        X_matrix_times_theta_miny = np.matmul(X_matrix,theta) - y
        theta = theta - np.dot(gamma_X_matrix_T,X_matrix_times_theta_miny)/n
        stop_condition = (np.abs(theta[1] - store_theta[-1][1])/store_theta[-1][1] < epsilon) and (np.abs(theta[0] - store_theta[-1][0])/store_theta[-1][0] < epsilon)
       # stop_condition = (((theta[1]-store_theta[-1][1])**2-(theta[0]-store_theta[-1][0])**2)**0.5/((store_theta[-1][1])**2-(store_theta[-1][0])**2)**0.5 < epsilon) 
        k +=1
    print(k)
    return(store_theta)


In [56]:
x_house = housing_data['median_house_value'].to_numpy()
y_house = housing_data['total_bedrooms'].to_numpy()
a = gradient_descent(x=x_house, y=y_house, gamma=0.0001, l=1)
b = gradient_descent(x=fakex, y=fakey, gamma=0.001, l=1)
C = gradient_descent(x=fakex, y=fakey, gamma=0.01, l=1)
print(b[-1],C[-1])

3771
1437
842
[-1.59130127  1.51117259] [-4.55669073  1.96028456]


<ipython-input-55-f3754f4a30c4>:35: RuntimeWarning: divide by zero encountered in double_scalars
  stop_condition = (np.abs(theta[1] - store_theta[-1][1])/store_theta[-1][1] < epsilon) and (np.abs(theta[0] - store_theta[-1][0])/store_theta[-1][0] < epsilon)


In [57]:
def plotfit(x, beta):
    funct = beta[-1][0]+x*beta[-1][1]
    return(funct)

def plotfit2(x, beta):
    functs=[]
    points = np.linspace(0,len(beta),10)
    points = [int(x) for x in points]
    for i in range(len(points)-1):
        functs.append(beta[points[i]][0]+x*beta[points[i]][1])
    return(functs)

def ColorGradient(start = (256,256,256), stop=(0,0,0), n=2):
    '''
    Pick two colour and gives back gradient of those colours
    '''
    gradR = np.linspace(start[0]/256,stop[0]/256,n)
    gradG = np.linspace(start[1]/256,stop[1]/256,n)
    gradB = np.linspace(start[2]/256,stop[2]/256,n)
    gradient = []
    for i in range(n):
        gradient.append((gradR[i],gradG[i],gradB[i]))


    return gradient

In [58]:
ifig=1;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
plt.scatter(fakex,fakey, c='darkred',  s=15)
nlines = 10
for i in range(nlines):
    plt.plot(fakex,plotfit2(fakex,beta=C)[i-1], color=ColorGradient(stop=(11, 0, 59), n=nlines)[i],lw=1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [74]:
ifig=2;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
plt.scatter(x_house, y=y_house, c='g',  s=15)
nlines = 10
for i in range(nlines):
    plt.plot(x_house,plotfit2(x_house,beta=a)[i-1],\
             color=ColorGradient(stop=(0, 10, 59), n=nlines)[i],lw=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [73]:
ifig=3;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
d = gradient_descent(x=year, y=tempa, gamma=0.01, l=1, theta_init = np.array([0.5,0.5]))
plt.scatter(year, y=tempa, c='orange',  s=15)
nlines = 10
for i in range(nlines):
    plt.plot(year,plotfit2(year,beta=d)[i-1],\
             color=ColorGradient(stop=(31, 80, 40), n=nlines)[i],lw=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

6703


In [65]:
ifig=4;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
plt.scatter(housing_data['median_house_value'], y=housing_data['median_income'], c='darkred',  s=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
 d=gradient_descent(x=fakex, y=fakey, gamma=0.0001, l=2)
 print(d[-1])

1853
[-0.13669904  0.00723493  0.16984273]


<ipython-input-9-a35c0fad958e>:32: RuntimeWarning: divide by zero encountered in double_scalars
  stop_condition = (np.abs(theta[1] - store_theta[-1][1])/store_theta[-1][1] < epsilon) and (np.abs(theta[0] - store_theta[-1][0])/store_theta[-1][0] < epsilon)
